In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
data_folder = "./data/"

df_legislaturas = pd.read_csv(data_folder + "df_legislaturas.csv")
df_parlamentares_por_legislatura = pd.read_csv(
    data_folder + "df_parlamentares_por_legislatura.csv"
)
df_parl_detalhes = pd.read_csv(data_folder + "df_parl_detalhes.csv", dtype={"cpf": str})
df_legis_lideres = pd.read_csv(data_folder + "df_legis_lideres.csv")
df_legis_mesa = pd.read_csv(data_folder + "df_legis_mesa.csv")
df_cargos_externos = pd.read_csv(data_folder + "df_cargos_externos.csv")
df_historico = pd.read_csv(data_folder + "df_historico.csv")
df_pareceres = pd.read_csv(data_folder + "df_pareceres.csv")
df_pareceres_detalhes = pd.read_csv(
    data_folder + "df_pareceres_detalhes.csv", low_memory=False
)
df_orgaos_membros = pd.read_csv(data_folder + "df_orgaos_membros.csv")
df_tse_candidatos = pd.read_csv(data_folder + "df_tse_candidatos.csv")

In [3]:
df_legislaturas["dataInicio"] = pd.to_datetime(df_legislaturas["dataInicio"])
df_legislaturas["dataFim"] = pd.to_datetime(df_legislaturas["dataFim"])

df_legislaturas["dataInicioYear"] = df_legislaturas["dataInicio"].dt.year
df_legislaturas["dataFimYear"] = df_legislaturas["dataFim"].dt.year

df_historico["dataHora"] = pd.to_datetime(df_historico["dataHora"])
# tratar cpfs
df_tse_candidatos["cpf"] = df_tse_candidatos["cpf"].astype(str).str.zfill(11)

In [4]:
df_parlamentares_por_legislatura = df_parlamentares_por_legislatura[
    ["id", "nome", "idLegislatura"]
].drop_duplicates()

# dimensão posição (mesa)

In [5]:
columns = ["id", "idLegislatura", "titulo", "codTitulo"]
group_columns = ["id", "idLegislatura", "titulo"]
df_legis_g = df_legis_mesa[columns].groupby(group_columns).count().reset_index()

In [6]:
posicao_mapper = {
    "Presidente": "mesa_1",
    "1º Vice-Presidente": "mesa_2",
    "2º Vice-Presidente": "mesa_2",
    "1º Secretário": "mesa_2",
    "2º Secretário": "mesa_2",
    "3º Secretário": "mesa_2",
    "4º Secretário": "mesa_2",
    # "2º Suplente de Secretário": "mesa_2",
    # "3º Suplente de Secretário": "mesa_2",
    # "4º Suplente de Secretário": "mesa_2",
    # "1º Suplente de Secretário": "mesa_2",
}

df_legis_g["posicao"] = df_legis_g["titulo"].apply(lambda x: posicao_mapper.get(x))

In [7]:
df_legis_final = df_legis_g.join(pd.get_dummies(df_legis_g["posicao"]))
# df_legis_final = df_legis_final[['id', 'idLegislatura', 'mesa_1', 'mesa_2']].reset_index(drop=True)
df_legis_final = (
    df_legis_final[["id", "idLegislatura", "mesa_1", "mesa_2"]]
    .groupby(["id", "idLegislatura"])
    .sum()
    .reset_index()
)

# dimensão posição (colégio de lideres)

In [8]:
df_legis_lideres_final = (
    df_legis_lideres[df_legis_lideres["titulo"].isin(["Líder", "Presidente"])]
    .drop_duplicates(["parlamentar.id", "parlamentar.idLegislatura", "titulo"])
    .groupby(["parlamentar.id", "parlamentar.idLegislatura"])
    .count()
    .reset_index()
    .rename(
        {
            "parlamentar.id": "id",
            "parlamentar.idLegislatura": "idLegislatura",
            "titulo": "pos_lider",
        },
        axis=1,
    )[["id", "idLegislatura", "pos_lider"]]
)

# dimensão presidencia de comissão

In [9]:
pos_comissao_mapper = {
    # "Coordenador": "pos_comiss_pr",
    # "Coordenadora": "pos_comiss_pr",
    # "Coordenador-Geral'": "pos_comiss_pr",
    "Presidente": "pos_comiss_pr",
}

In [10]:
df_orgaos_membros["titulo_trat"] = df_orgaos_membros["titulo"].apply(
    lambda x: pos_comissao_mapper.get(x)
)

In [11]:
df_orgaos_membros_final = (
    df_orgaos_membros[~df_orgaos_membros["titulo_trat"].isnull()]
    .drop_duplicates(["id_org", "id", "idLegislatura"])[
        ["id_org", "id", "idLegislatura"]
    ]
    .groupby(["id", "idLegislatura"])
    .count()
    .reset_index()
    .rename({"id_org": "pos_comiss_pr"}, axis=1)
)

# pareceres de relator

In [12]:
df_pareceres_merge = df_pareceres[["id", "id_parl"]].merge(
    df_pareceres_detalhes, on="id"
)

## encontrar legislatura por data de apresentação

In [13]:
df_pareceres_merge["dataApresentacao"] = pd.to_datetime(
    df_pareceres_merge["dataApresentacao"]
)

In [14]:
def find_legislatura(date):
    legislatura = df_legislaturas[
        (df_legislaturas["dataInicio"] <= date) & (df_legislaturas["dataFim"] >= date)
    ]
    if not legislatura.empty:
        return int(legislatura.iloc[0]["id"])
    return None

In [15]:
df_pareceres_merge["idLegislatura"] = df_pareceres_merge["dataApresentacao"].apply(
    find_legislatura
)
df_pareceres_merge["idLegislatura"] = (
    df_pareceres_merge["idLegislatura"].fillna(-1).astype("int64")
)

In [16]:
df_pareceres_final = (
    df_pareceres_merge[["id", "id_parl", "idLegislatura"]]
    .groupby(["id_parl", "idLegislatura"])
    .count()
    .reset_index()
    .rename({"id": "relatorias", "id_parl": "id"}, axis=1)
)

In [17]:
df_pareceres_final["relatorias_ln"] = np.log(df_pareceres_final["relatorias"])

# mandatos externos

In [18]:
df_mandatos_externos = pd.merge(
    df_parlamentares_por_legislatura,
    df_legislaturas[["id", "dataInicioYear"]],
    left_on="idLegislatura",
    right_on="id",
    how="left",
)

df_mandatos_externos.drop(columns=["id_y"], inplace=True)
df_mandatos_externos.rename(columns={"id_x": "id"}, inplace=True)

df_mandatos_externos["mand_dep_estadual"] = 0
df_mandatos_externos["mand_ver"] = 0
df_mandatos_externos["mand_sen"] = 0


def update_dim_columns(row, df_cargos, df_legislaturas):
    leg_data_inicio = df_legislaturas.loc[
        df_legislaturas["id"] == row["idLegislatura"], "dataInicio"
    ].dt.year.values[0]

    relevant_cargos = df_cargos[
        (df_cargos["id_parl"] == row["id"]) & (df_cargos["anoInicio"] < leg_data_inicio)
    ]

    for _, cargo in relevant_cargos.iterrows():
        if cargo["cargo_trat"] == "mand_dep_estadual":
            row["mand_dep_estadual"] += 1
        elif cargo["cargo_trat"] == "mand_ver":
            row["mand_ver"] += 1
        elif cargo["cargo_trat"] == "mand_sen":
            row["mand_sen"] += 1

    return row

In [19]:
def find_mandato(x):
    texto = x.lower()
    if "suplente" in texto:
        return None
    elif "estadual" in texto:
        return "mand_dep_estadual"
    elif "veread" in texto:
        return "mand_ver"
    elif "senad" in texto:
        return "mand_sen"
    else:
        return None


df_cargos_externos["cargo_trat"] = df_cargos_externos["cargo"].apply(
    lambda x: find_mandato(x)
)
df_cargos_externos["anos_mandato"] = (
    df_cargos_externos["anoFim"] - df_cargos_externos["anoInicio"]
)

In [20]:
df_cargos_externos_clean = df_cargos_externos[
    ~df_cargos_externos["cargo_trat"].isnull()
]

df_mandatos_externos = df_mandatos_externos.apply(
    update_dim_columns, axis=1, args=(df_cargos_externos_clean, df_legislaturas)
)

df_mandatos_externos.drop(columns=["dataInicioYear", "nome"], inplace=True)

# dim tempo de atuação na camara

In [21]:
def compute_period_in_ex(row, df_historico, df_legislaturas):
    id_parl = row["id"]
    id_legis = row["idLegislatura"]

    current_legis_row = df_legislaturas[
        df_legislaturas["dataFim"] > datetime.datetime.today()
    ].iloc[0]
    id_legislatura_atual = current_legis_row["id"]

    legis_row = df_legislaturas[df_legislaturas["id"] == id_legis].iloc[0]
    date_init_legis = legis_row["dataInicio"]
    date_end_legis = legis_row["dataFim"]

    df_hist_parl = df_historico[
        (df_historico["id"] == id_parl)
        & (df_historico["idLegislatura"] == id_legis)
        & (
            df_historico["dataHora"].dt.floor("D")
            >= np.datetime64(date_init_legis, "D")
        )
        & (df_historico["dataHora"].dt.floor("D") <= np.datetime64(date_end_legis, "D"))
    ].sort_values("dataHora")

    period_in_ex = 0
    last_data_hora_ex = None
    in_ex = False

    for _, row_hist in df_hist_parl.iterrows():
        status = row_hist["situacao"]
        data_hora = row_hist["dataHora"]

        if status == "Exercício" or (str(status) == "nan" and id_legis < 50):
            if not in_ex:
                last_data_hora_ex = data_hora
                in_ex = True
        else:
            if in_ex and status != "Convocado":
                period_in_ex += (data_hora - last_data_hora_ex).days
                in_ex = False

    if id_legis == id_legislatura_atual:
        total_days = (datetime.datetime.today() - date_init_legis).days - 32
        if in_ex:
            period_in_ex += (datetime.datetime.today() - last_data_hora_ex).days
    else:
        total_days = (date_end_legis - date_init_legis).days

    if in_ex and period_in_ex == 0:
        period_in_ex = (data_hora - last_data_hora_ex).days

    percent_period_in_ex = round(period_in_ex / total_days, 2)
    row["tempo_atuacao_dias"] = period_in_ex
    row["tempo_atuacao_percent"] = percent_period_in_ex
    # row["last_data_hora_ex"] = last_data_hora_ex
    # row["last_data_hora"] = data_hora
    # row["in_ex"] = in_ex
    return row

In [22]:
df_tempo_atuacao = df_parlamentares_por_legislatura.copy()

df_tempo_atuacao = df_tempo_atuacao.apply(
    compute_period_in_ex, axis=1, args=(df_historico, df_legislaturas)
)

df_tempo_atuacao.drop(columns=["nome"], inplace=True)

# dim fidelidade

In [23]:
df_fidelidade = pd.merge(
    df_parlamentares_por_legislatura,
    df_legislaturas[["id", "dataInicioYear", "dataFimYear"]],
    left_on="idLegislatura",
    right_on="id",
    how="left",
)

df_fidelidade.drop(columns=["id_y"], inplace=True)
df_fidelidade.rename(columns={"id_x": "id"}, inplace=True)

df_fidelidade = df_fidelidade.merge(df_parl_detalhes[["id", "cpf"]], on="id")

df_fidelidade["fid_municipais"] = 0
df_fidelidade["fid_gerais"] = 0

In [24]:
def update_fid_columns(row, df_tse):
    cargos_municipais = ["vereador", "prefeito", "vice-prefeito"]
    cargos_gerais_congresso = [
        # "1º suplente senador",
        "senador",
        "deputado federal",
        # "2º suplente senador",
        # "1º suplente",
        # "2º suplente",
    ]
    eleicoes_municipais = df_tse[
        (df_tse["cargo"].isin(cargos_municipais))
        & (df_tse["ano"] > row["dataInicioYear"])
        & (df_tse["ano"] < row["dataFimYear"])
    ]

    eleicoes_cn = df_tse[
        (df_tse["cargo"].isin(cargos_gerais_congresso))
        & (df_tse["ano"] > row["dataInicioYear"])
        & (df_tse["ano"] < row["dataFimYear"])
    ]

    if row["cpf"] not in eleicoes_municipais["cpf"].values:
        row["fid_municipais"] = 1

    if row["cpf"] in eleicoes_cn["cpf"].values:
        row["fid_gerais"] = 1

    return row

In [25]:
df_fidelidade = df_fidelidade.apply(
    update_fid_columns, axis=1, args=(df_tse_candidatos,)
)
df_fidelidade.drop(
    columns=["dataInicioYear", "dataFimYear", "nome", "cpf"], inplace=True
)

# dim idade da primeira posse

In [26]:
df_idade_posse = df_parlamentares_por_legislatura[["id", "idLegislatura"]].merge(
    df_parl_detalhes[
        [
            "id",
            "dataNascimento",
        ]
    ],
    on="id",
    how="left",
)

In [27]:
df_idade_posse_merge = df_idade_posse.merge(
    df_legislaturas[["id", "dataInicio"]], left_on="idLegislatura", right_on="id"
)
df_idade_posse_merge["dataNascimento"] = pd.to_datetime(
    df_idade_posse_merge["dataNascimento"]
)
df_idade_posse_merge["idade_posse"] = (
    df_idade_posse_merge["dataInicio"] - df_idade_posse_merge["dataNascimento"]
)
df_idade_posse_merge["idade_posse"] = df_idade_posse_merge["idade_posse"].apply(
    lambda x: int(x.days / 365.25) if pd.notna(x) else None
)
df_idade_posse_merge = (
    df_idade_posse_merge[["id_x", "idade_posse"]]
    .groupby(["id_x"])
    .min()
    .reset_index()
    .rename({"id_x": "id"}, axis=1)
)

# merge dims

In [28]:
df_final = (
    df_parlamentares_por_legislatura[["id", "idLegislatura"]]
    .merge(
        df_parl_detalhes[
            [
                "id",
                "ultimoStatus.nome",
                "ultimoStatus.siglaPartido",
                "sexo",
                "dataNascimento",
                "escolaridade",
            ]
        ],
        on="id",
        how="left",
    )
    .merge(df_idade_posse_merge, on=["id"], how="left")
    .merge(df_tempo_atuacao, on=["id", "idLegislatura"], how="left")
    .merge(df_legis_final, on=["id", "idLegislatura"], how="left")
    .merge(df_legis_lideres_final, on=["id", "idLegislatura"], how="left")
    .merge(df_orgaos_membros_final, on=["id", "idLegislatura"], how="left")
    .merge(df_pareceres_final, on=["id", "idLegislatura"], how="left")
    .merge(df_mandatos_externos, on=["id", "idLegislatura"], how="left")
    .merge(df_fidelidade, on=["id", "idLegislatura"], how="left")
    .fillna(0)
    .drop_duplicates()
)

In [29]:
def accumulate(row, df):
    columns_to_cumsum = [
        "mesa_1",
        "mesa_2",
        "pos_lider",
        "pos_comiss_pr",
        "fid_municipais",
        "fid_gerais",
        "relatorias"
    ]

    columns_to_avg = ["tempo_atuacao_percent"]

    columns_to_do_nothing = ["mand_dep_estadual", "mand_ver", "mand_sen"]

    id_parl = row["id"]
    id_legis = row["idLegislatura"]

    df_filtered = df[(df["id"] == id_parl) & (df["idLegislatura"] <= id_legis)]

    for col in columns_to_cumsum:
        row[f"{col}_cum"] = df_filtered[col].sum()

    for col in columns_to_avg:
        row[f"{col}_cum"] = df_filtered[col].mean()

    for col in columns_to_do_nothing:
        row[f"{col}_cum"] = row[col]

    return row

def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())


In [30]:
df = df_final.copy().sort_values(["id", "idLegislatura"])
df_res = df.apply(
    accumulate, axis=1, args=(df,)
)

df_res['relatorias_ln_cum'] = np.log(df_res['relatorias_cum'] + 1)


In [31]:
columns_to_norm = [
    "idade_posse",
    "mesa_1_cum",
    "mesa_2_cum",
    "pos_lider_cum",
    "pos_comiss_pr_cum",
    "fid_municipais_cum",
    "fid_gerais_cum",
    "tempo_atuacao_percent_cum",
    "mand_dep_estadual_cum",
    "mand_ver_cum",
    "mand_sen_cum",
    "relatorias_ln_cum"
]

for col in columns_to_norm:
    df_res[f'{col}_norm'] = min_max_normalize(df_res[col])

df_res['idade_posse_norm'] = 1 - df_res['idade_posse_norm'] 

In [32]:
# appply weights
weights = {
    "idade_posse_norm": 1,
    "tempo_atuacao_percent_cum_norm": 1,
    "mesa_1_cum_norm": 5,
    "mesa_2_cum_norm": 3,
    "pos_lider_cum_norm": 4,
    "pos_comiss_pr_cum_norm": 3,
    "relatorias_ln_cum_norm": 2,
    "mand_dep_estadual_cum_norm": 0.2,
    "mand_ver_cum_norm": 0.1,
    "mand_sen_cum_norm": 0.7,
    "fid_municipais_cum_norm": 1,
    "fid_gerais_cum_norm": 1,
}
for var_name, weight in weights.items():
    df_res[f"w_{var_name}"] = df_res[var_name] * weight



In [33]:
df_res['dim_comprometimento'] = (
    df_res['w_tempo_atuacao_percent_cum_norm'] +
    df_res['w_mesa_1_cum_norm'] +
    df_res['w_mesa_2_cum_norm'] +
    df_res['w_pos_lider_cum_norm'] +
    df_res['w_pos_comiss_pr_cum_norm'] +
    df_res['w_relatorias_ln_cum_norm']
)

df_res['dim_carreira'] = (
    df_res['w_idade_posse_norm'] +
    df_res['w_mand_dep_estadual_cum_norm'] +
    df_res['w_mand_ver_cum_norm'] +
    df_res['w_mand_sen_cum_norm']
)
df_res['dim_fidelidade'] = (
    df_res['w_fid_municipais_cum_norm'] +
    df_res['w_fid_gerais_cum_norm']
)

In [34]:
for col in ['dim_comprometimento', 'dim_carreira', 'dim_fidelidade']:
    df_res[f'{col}_norm'] = min_max_normalize(df_res[col])

In [35]:
df_res['ipp'] = (df_res['dim_comprometimento_norm'] + df_res['dim_carreira_norm'] + df_res['dim_fidelidade_norm']) / 3

In [36]:
df_res.sort_values('ipp')

,id,idLegislatura,ultimoStatus.nome,ultimoStatus.siglaPartido,sexo,dataNascimento,escolaridade,idade_posse,tempo_atuacao_dias,tempo_atuacao_percent,...,w_mand_sen_cum_norm,w_fid_municipais_cum_norm,w_fid_gerais_cum_norm,dim_comprometimento,dim_carreira,dim_fidelidade,dim_comprometimento_norm,dim_carreira_norm,dim_fidelidade_norm,ipp
7425,194258,55,CREUZA PEREIRA,PSB,F,1936-07-10,Ensino Médio,78.0,670,0.46,...,0.0,0.000000,0.000000,0.859558,0.060241,0.000000,0.085813,0.059546,0.000000,0.048453
5423,221149,56,Elói Pietá,PT,M,1944-08-10,Superior,74.0,30,0.02,...,0.0,0.000000,0.000000,0.018868,0.153672,0.000000,0.001884,0.151898,0.000000,0.051261
2511,179382,54,MANOEL ANTUNES,PDT,M,1934-03-01,Superior,76.0,30,0.02,...,0.0,0.000000,0.142857,0.018868,0.084337,0.142857,0.001884,0.083364,0.071429,0.052225
1080,139312,48,MOYSÉS PIMENTEL,PMDB,M,1909-08-11,Secundário,77.0,1460,1.00,...,0.0,0.111111,0.000000,0.943396,0.072289,0.111111,0.094183,0.071455,0.055556,0.073731
2663,160516,54,RAIMUNDÃO,PMDB,M,1942-11-30,Superior,68.0,570,0.39,...,0.0,0.000000,0.000000,0.608205,0.180723,0.000000,0.060719,0.178637,0.000000,0.079786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,73433,55,Arlindo Chinaglia,PT,M,1949-12-24,Pós-Graduação,45.0,1460,1.00,...,0.0,0.666667,0.857143,8.396849,0.486403,1.523810,0.838287,0.480789,0.761905,0.693660
8255,74210,55,SARNEY FILHO,PV,M,1957-06-14,Superior,29.0,673,0.46,...,0.0,0.888889,0.857143,5.769695,0.679174,1.746032,0.576009,0.671336,0.873016,0.706787
5231,73433,56,Arlindo Chinaglia,PT,M,1949-12-24,Pós-Graduação,45.0,1460,1.00,...,0.0,0.777778,1.000000,9.200412,0.486403,1.777778,0.918510,0.480789,0.888889,0.762729
6032,74693,56,Rodrigo Maia,PSDB,M,1970-06-12,Superior Incompleto,28.0,1094,0.75,...,0.0,0.555556,0.714286,10.016675,0.662651,1.269841,1.000000,0.655003,0.634921,0.763308


In [37]:
df_res.to_excel("df_res.xlsx", index=False)

# Descritivas

In [41]:
df_parlamentares_por_legislatura.groupby('idLegislatura').count()

,id,nome
idLegislatura,,
48,589,589
49,620,613
50,635,634
51,642,638
52,658,656
53,664,664
54,684,684
55,649,649
56,637,637


In [51]:
df_pareceres_detalhes['dataApresentacao'].min()

'1983-08-17T15:10'

In [54]:
df_orgaos_membros['dataInicio'].min()

'1999-02-01'

In [73]:
tot_parls = len(df_parl_detalhes['id'].unique())
tot_parls_tse = len(df_parl_detalhes.merge(df_tse_candidatos, on='cpf')['id'].unique())

print(tot_parls, tot_parls_tse, tot_parls_tse / tot_parls)


3123 2687 0.8603906500160102


In [70]:
len(df_parl_detalhes.merge(df_tse_candidatos, on='cpf')['id'].unique())

2687

In [86]:
print(len(df_legis_lideres))
print(len(df_legis_mesa))
print(len(df_cargos_externos))
print(len(df_historico))
print(len(df_pareceres['id'].unique()))
print(len(df_orgaos_membros))

11330
133
4531
23997
112984
81974


In [97]:
print(df_final[
    [
        "idade_posse",
        "tempo_atuacao_dias",
        "tempo_atuacao_percent",
        "mesa_1",
        "mesa_2",
        "pos_lider",
        "pos_comiss_pr",
        "relatorias",
        "relatorias_ln",
        "mand_dep_estadual",
        "mand_ver",
        "mand_sen",
        "fid_municipais",
        "fid_gerais",
    ]
].describe().T.to_latex(float_format='%.2f'))

\begin{tabular}{lrrrrrrrr}
\toprule
 & count & mean & std & min & 25% & 50% & 75% & max \\
\midrule
idade_posse & 6250.00 & 44.99 & 9.80 & 0.00 & 38.00 & 45.00 & 51.00 & 83.00 \\
tempo_atuacao_dias & 6250.00 & 1204.77 & 438.50 & 0.00 & 1031.25 & 1460.00 & 1460.00 & 1460.00 \\
tempo_atuacao_percent & 6250.00 & 0.88 & 0.27 & 0.00 & 0.98 & 1.00 & 1.00 & 1.06 \\
mesa_1 & 6250.00 & 0.00 & 0.05 & 0.00 & 0.00 & 0.00 & 0.00 & 1.00 \\
mesa_2 & 6250.00 & 0.01 & 0.12 & 0.00 & 0.00 & 0.00 & 0.00 & 2.00 \\
pos_lider & 6250.00 & 0.06 & 0.24 & 0.00 & 0.00 & 0.00 & 0.00 & 2.00 \\
pos_comiss_pr & 6250.00 & 0.13 & 0.38 & 0.00 & 0.00 & 0.00 & 0.00 & 4.00 \\
relatorias & 6250.00 & 18.07 & 123.05 & 0.00 & 0.00 & 4.00 & 17.00 & 5842.00 \\
relatorias_ln & 6250.00 & 1.52 & 1.54 & 0.00 & 0.00 & 1.39 & 2.83 & 8.67 \\
mand_dep_estadual & 6250.00 & 0.61 & 1.06 & 0.00 & 0.00 & 0.00 & 1.00 & 7.00 \\
mand_ver & 6250.00 & 0.39 & 0.77 & 0.00 & 0.00 & 0.00 & 1.00 & 6.00 \\
mand_sen & 6250.00 & 0.03 & 0.19 & 0.00 & 0.00

In [90]:
df_final.columns

Index(['id', 'idLegislatura', 'ultimoStatus.nome', 'ultimoStatus.siglaPartido',
       'sexo', 'dataNascimento', 'escolaridade', 'idade_posse',
       'tempo_atuacao_dias', 'tempo_atuacao_percent', 'mesa_1', 'mesa_2',
       'pos_lider', 'pos_comiss_pr', 'relatorias', 'relatorias_ln',
       'mand_dep_estadual', 'mand_ver', 'mand_sen', 'fid_municipais',
       'fid_gerais'],
      dtype='object')